In [93]:
import nltk
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PARKERGRIEP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PARKERGRIEP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [94]:
import json
import string
import random
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import torch
from torch import nn
from textblob import Word
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [95]:
with open("AllPrintings.json", encoding='utf-8') as f:
    data = json.load(f)

In [96]:
cards = {}
for group_key in list(data["data"].keys()):
    for card in data["data"][group_key]["cards"]:
        if "foreignData" in list(card.keys()):
            del card["foreignData"]
        name = card["name"]
        cards[name] = card

In [97]:
def two_or_fewer_colors(text):
    shared_characters = ''.join(set("RBGUW").intersection(text)) 
    return len(shared_characters) in [1,2]

def filter_card(key):    
    return ("text" in list(cards[key].keys()) 
            and "//" not in key 
            and cards[key]["rarity"] not in ["bonus", "special"]
            and any([t in cards[key]["type"] for t in ["Planeswalker", "Enchantment", "Sorcery", "Instant", "Artifact", "Creature"]])
            and "Land" not in cards[key]["type"]
            and "manaCost" in list(cards[key].keys())
            and two_or_fewer_colors(cards[key]["colors"])
            #below names have no mana_cost, strangely
            and key not in ["Pact of Negation", "Summoner's Pact", "Rograkh, Son of Rohgahh", "Intervention Pact", "Slaughter Pact", "Pact of the Titan"])
    
cards = {key : cards[key]
         for key in list(cards.keys())
         if filter_card(key)}
print(len(list(cards.keys())))
card_texts = {key: cards[key]["text"] for key in list(cards.keys())}

17742


In [98]:
def process_color(text):
    colors = []
    for c in text.split("{"):
        c = c.replace("}", "")
        if c.isdigit():
            n_blank = int(c)
            colors.append(n_blank*"M")
        else:
            colors.append(c)
    colors = [c for c in colors if c != '']
    colors.sort()
    s = "".join(colors)
    return f"{s}"

def process_before_colon(text):
    length_check = len(text.split(" ")) >= 6
    if length_check:
        text = text.replace(",", "")
    precolon_elements = ["<precolon>"]
    for word in text.split(" "):
        if "{" in word:
            precolon_elements.append("<color>")
            precolon_elements.append(process_color(word))
            precolon_elements.append("</color>")
        elif "[" in word:
            word = word.replace("[", "")
            word = word.replace("]", "")
            precolon_elements.append("<symbol>")
            precolon_elements.append(word)
            precolon_elements.append("</symbol>")
        else:
            precolon_elements.append(add_tags(word))
    precolon_elements.append("</precolon>")
    return " ".join(precolon_elements)
            

def add_tags(text):
    
    sections = text.split("(")
    if len(sections) > 2 or (len(sections) == 2 
                             and sections[1].index(")") not in [len(sections[1])-1,len(sections[1])-2]):
        raise ValueError(text)
    elif len(sections) == 2:
        second_section = sections[1].replace(")", "")
        return f"{add_tags(sections[0])} <reminder> {add_tags(second_section)} </reminder>"
    
    if ":" in text:
        sections = text.split(":")
        return f"{process_before_colon(sections[0])} {add_tags(sections[1])}"
    
    tags = []
    if "." in text:
        tags.append("<sentence>")
        
    color_or_tag = False
    for word in text.split(" "):
        if word in ["<color>", "<symbol>", "</color>", "</symbol>"]:
            color_or_tag = not color_or_tag
        if "." in word:
            word = word.replace(".", "")
        if "<" not in word and not color_or_tag:
            word = word.lower()
        tags.append(word)
    if "." in text:
        tags.append("</sentence>")
    
    return " ".join(tags)

In [99]:
stop = stopwords.words('English')
punctuation = [x for x in string.punctuation if x not in "+/,<>_{}:.'\""]
lemmatized_card_texts = {}
for index, key in tqdm(enumerate(list(card_texts.keys()))):
    t = card_texts[key]
    t = t.replace(key, "<CARD_NAME>")
    t = t.replace('"', "\n")
    t = t.strip()
    t = "".join([char for char in list(t) if char not in punctuation])
    print(key)
    t = [add_tags(line) for line in t.split("\n")]
    tags = ["<card>"]
    for index, line in enumerate(t):
        line_tag = "line" if "•" not in line else "bullet"
        tags.append(f"<{line_tag}>")
        tags.append(line.replace("•", ""))
        tags.append(f"</{line_tag}>")
    tags.append("</card>")
    t = " ".join(tags)
    t = t.replace("<card_name>", "<CARD_NAME>")
    t = t.replace("<CARD_NAME>s", "<CARD_NAME>")
    t.replace(",", "")
    t = str(Word(t).lemmatize())
    lemmatized_card_texts[str(key)] = t
    
    t = "".join([char.lower() for char in list(t) if char not in punctuation])
    t = t.replace('[^\w\s]','')
    #t = " ".join([word.lower() for word in t.split(" ") if word not in stop])
#     t = " ".join([word.lower() for word in t.split(" ")])
#     t = str(Word(t).lemmatize())
#     lemmatized_card_texts.append(t.split(" "))

# model = Word2Vec(lemmatized_card_texts, min_count=1,size=75,workers=4)
# model.save("word2vec.model")
# model.save("model.bin")

Ancestor's Chosen
Angel of Mercy
Aven Cloudchaser
Ballista Squad
Bandage
Beacon of Immortality
Benalish Knight
Cho-Manno, Revolutionary
Condemn
Contaminated Bond
Drudge Skeletons
Demystify
Fear
Field Marshal
Hypnotic Specter
Ghost Warden
Glorious Anthem
Hail of Arrows
Heart of Light
High Ground
Honor Guard
Icatian Priest
Reviving Dose
Reya Dawnbringer
Righteousness
Samite Healer
Serra's Embrace
Academy Researchers
Air Elemental
Ambassador Laquatus
Aven Fisher
Aven Windreader
Boomerang
Cancel
Cephalid Constable
Clone
Cloud Elemental
Cloud Sprite
Counsel of the Soratami
Crafty Pathmage
Dehydration
Deluge
Denizen of the Deep
Discombobulate
Dreamborn Muse
Evacuation
Afflict
Agonizing Memories
Ascendant Evincar
Assassinate
Beacon of Unrest
Bog Wraith
Hurkyl's Recall
Reminisce
Remove Soul
Robe of Mirrors
Rootwater Commando
Rule of Law
Rootwater Matriarch
Sage Owl
Scalpelexis
Serra Angel
Sea Monster
Shimmering Wings
Sift
Sky Weaver
Mortivore
Nomad Mythmaker
Paladin en-Vec
Peek
Phantom Warrior

Iron-Barb Hellion
Krark-Clan Engineers
Krark-Clan Ogre
Magma Giant
Magma Jet
Magnetic Theft
Mana Geyser
Rain of Rust
Reversal of Fortune
Screaming Fury
Vulshok Sorcerer
All Suns' Dawn
Beacon of Creation
Bringer of the Green Dawn
Channel the Suns
Dawn's Reflection
Eternal Witness
Fangren Pathcutter
Ferocious Charge
Ouphe Vandals
Rite of Passage
Rude Awakening
Sylvok Explorer
Tangle Asp
Tel-Jilad Justice
Tel-Jilad Lifebreather
Tornado Elemental
Tyrranax
Viridian Lorebearers
Viridian Scout
Abbey Gargoyles
Caribou Range
Initiates of the Ebon Hand
Akron Legionnaire
Arenson's Aura
Armor of Faith
Aysen Bureaucrats
Blessed Wine
Blinking Spirit
D'Avenant Archer
Death Speakers
Divine Offering
Dust to Dust
Greater Realm of Preservation
Heal
Hipparion
Icatian Phalanx
Icatian Scout
Icatian Town
Ivory Guardians
Justice
Kjeldoran Skycaptain
Mesa Falcon
Order of the Sacred Torch
Order of the White Shield
Prismatic Ward
Repentant Blacksmith
Sacred Boon
Seraph
Serra Bestiary
Serra Paladin
Shield Bearer


Efficient Construction
Hinterland Drake
Ice Over
Illusionist's Stratagem
Leave in the Dust
Mechanized Production
Negate
Quicksmith Spy
Reverse Engineer
Salvage Scuttler
Shielded Aether Thief
Shipwreck Moray
Skyship Plunderer
Take into Custody
Trophy Mage
Whir of Invention
Wind-Kin Raiders
Aether Poisoner
Battle at the Bridge
Cruel Finality
Daring Demolition
Fen Hauler
Foundry Hornet
Fourth Bridge Prowler
Gifted Aetherborn
Glint-Sleeve Siphoner
Gonti's Machinations
Herald of Anguish
Ironclad Revolutionary
Midnight Entourage
Night Market Aeronaut
Perilous Predicament
Renegade's Getaway
Resourceful Return
Secret Salvage
Sly Requisitioner
Vengeful Rebel
Yahenni, Undying Partisan
Yahenni's Expertise
Aether Chaser
Chandra's Revolution
Destructive Tampering
Embraal Gear-Smasher
Enraged Giant
Freejam Regent
Frontline Rebel
Gremlin Infestation
Hungry Flames
Indomitable Creativity
Invigorated Rampage
Kari Zev, Skyship Raider
Kari Zev's Expertise
Lightning Runner
Pia's Revolution
Precise Strike
Q

Ooze Garden
Resounding Roar
Rhox Charger
Sacellum Godspeaker
Savage Hunger
Soul's Might
Spearbreaker Behemoth
Topan Ascetic
Wild Nacatl
Agony Warp
Ajani Vengeant
Blood Cultist
Branching Bolt
Bull Cerodon
Deft Duelist
Goblin Deathraiders
Hindering Light
Jhessian Infiltrator
Kiss of the Amesha
Necrogenesis
Qasali Ambusher
Rip-Clan Crasher
Sangrite Surge
Sarkhan Vol
Sigil Blessing
Steward of Valeron
Swerve
Thoughtcutter Agent
Tidehollow Strix
Taste of Paradise
Undergrowth
Whip Vine
Yavimaya Ancients
Martyrdom
Noble Steeds
Reinforcements
Royal Decree
Royal Herbalist
Scars of the Veteran
Seasoned Tactician
Sustaining Spirit
Sworn Defender
Unlikely Alliance
Spiny Starfish
Yavimaya Ants
Carrier Pigeons
Wild Aesthir
Awesome Presence
Benthic Explorers
False Demise
Foresight
Errand of Duty
Lat-Nam's Legacy
Inheritance
Ivory Gargoyle
Balduvian Dead
Casting of Bones
Contagion
Diseased Vermin
Dystopia
Fatal Lore
Feast or Famine
Fevered Strength
Insidious Bookworms
Juniper Order Advocate
Phantasmal 

Rowan Kenrith
Regna, the Redeemer
Krav, the Unredeemed
Zndrsplt, Eye of Wisdom
Okaun, Eye of Chaos
Virtus the Veiled
Gorm the Great
Khorvath Brightflame
Sylvia Brightspear
Pir, Imaginative Rascal
Toothy, Imaginary Friend
Blaring Recruiter
Blaring Captain
Chakram Retriever
Chakram Slinger
Soulblade Corrupter
Soulblade Renewer
Impetuous Protege
Proud Mentor
Ley Weaver
Lore Weaver
Arena Rector
Aurora Champion
Brightling
Bring Down
Dwarven Lightsmith
Jubilant Mascot
Play of the Game
Regna's Sanction
Skystreamer
Together Forever
Arcane Artisan
Fumble
Game Plan
Huddle Up
Nimbus Champion
Out of Bounds
Saltwater Stalwart
Soaring Show-Off
Spellseeker
Spellweaver Duo
Zndrsplt's Judgment
Archfiend of Despair
Bloodborn Scoundrels
Fan Favorite
Gang Up
Inner Demon
Mindblade Render
Sickle Dancer
Stunning Reversal
Thrasher Brute
Thrilling Encore
Virtus's Maneuver
Azra Bladeseeker
Bonus Round
Bull-Rush Bruiser
Cheering Fanatic
Khorvath's Fury
Lava-Field Overlord
Magma Hellion
Najeela, the Blade-Blossom

...

Open the Gates
Rampaging Rendhorn
Regenesis
Sagittars' Volley
Saruli Caretaker
Sauroform Hybrid
Steeple Creeper
Territorial Boar
Titanic Brawl
Trollbred Guardian
Wrecking Beast
Aeromunculus
Applied Biomancy
Azorius Knight-Arbiter
Azorius Skyguard
Basilica Bell-Haunt
Bolrac-Clan Crusher
Captive Audience
Clan Guildmage
Combine Guildmage
Cult Guildmage
Dovin, Grand Arbiter
Dovin's Acuity
Final Payment
Fireblade Artist
Frenzied Arynx
Frilled Mystic
Galloping Lizrog
Get the Point
Grasping Thrull
Gyre Engineer
Hackrobat
High Alert
Imperious Oligarch
Knight of the Last Breath
Macabre Mockery
Pitiless Pontiff
Prime Speaker Vannifar
Rafter Demon
Rakdos Roustabout
Rhythm of the Wild
Rubblebelt Runner
Savage Smash
Senate Guildmage
Sphinx of New Prahv
Sphinx's Insight
Sunder Shaman
Syndicate Guildmage
Zhur-Taa Goblin
Footlight Fiend
Rubble Slinger
Scuttlegator
Senate Griffin
Vizkopa Vampire
Dovin, Architect of Law
Elite Arrester
Dovin's Dismissal
Domri, City Smasher
Ragefire
Charging War Boar
Domr

Kaho, Minamo Historian
Kiri-Onna
Meishin, the Mind Cage
Moonbow Illusionist
Oboro Breezecaller
Oboro Envoy
Oppressive Will
Rushing-Tide Zubera
Secretkeeper
Shape Stealer
Shifting Borders
Shinen of Flight's Wings
Soramaro, First to Dream
Trusted Advisor
Akuta, Born of Ash
Choice of Damnations
Death of a Thousand Stings
Deathknell Kami
Deathmask Nezumi
Exile into Darkness
Footsteps of the Goryo
Gnat Miser
Hand of Cruelty
Infernal Kirin
Kagemaro, First to Suffer
Kagemaro's Clutch
Kami of Empty Graves
Kemuri-Onna
Kiku's Shadow
Kuro's Taken
Locust Miser
Maga, Traitor to Mortals
Measure of Wickedness
Neverending Torment
Pain's Reward
Raving Oni-Slave
Shinen of Fear's Chill
Sink into Takenuma
Skull Collector
Adamaro, First to Desire
Akki Drillmaster
Akki Underling
Barrel Down Sokenzan
Burning-Eye Zubera
Gaze of Adamaro
Glitterfang
Godo's Irregulars
Iizuka the Ruthless
Inner Fire
Into the Fray
Jiwari, the Earth Aflame
Path of Anger's Flame
Rally the Horde
Ronin Cavekeeper
Skyfire Kirin
Sokenza

Spear of Heliod
Vanquish the Foul
Wingsteed Rider
Artisan of Forms
Breaching Hippocamp
Coastline Chimera
Crackling Triton
Fate Foretold
Lost in a Labyrinth
Meletis Charlatan
Nimbus Naiad
Ordeal of Thassa
Sea God's Revenge
Stymied Hopes
Thassa's Bounty
Thassa's Emissary
Triton Fortune Hunter
Wavecrash Triton
Agent of the Fates
Asphodel Wanderer
Baleful Eidolon
Boon of Erebos
Cavern Lampad
Cutthroat Maneuver
Dark Betrayal
Disciple of Phenax
Erebos's Emissary
Fleshmad Steed
Hythonia the Cruel
Insatiable Harpy
Lash of the Whip
Loathsome Catoblepas
March of the Returned
Mogis's Marauder
Ordeal of Erebos
Pharika's Cure
Scourgemark
Sip of Hemlock
Viper's Kiss
Akroan Crusader
Arena Athlete
Boulderfall
Deathbellow Raider
Firedrinker Satyr
Flamespeaker Adept
Labyrinth Champion
Messenger's Speed
Peak Eruption
Priest of Iroas
Purphoros's Emissary
Rage of Purphoros
Satyr Rambler
Spark Jolt
Spearpoint Oread
Stormbreath Dragon
Two-Headed Cerberus
Agent of Horizons
Artisan's Sorrow
Centaur Battlemaste

Multani's Presence
Repopulate
Silk Net
Simian Grunts
Treefolk Mystic
Weatherseed Elf
Weatherseed Treefolk
Yavimaya Granger
Angelic Renewal
Back to Basics
Adorable Kitten
AWOL
Emcee
Flavor Judge
Frankie Peanuts
GO TO JAIL
Look at Me, I'm R&D
Old Guard
Ordinary Pony
Staying Power
Strutting Turkey
Syr Cadian, Knight Owl
Alexander Clamilton
Avatar of Me
B.O.B. (Bevy of Beebles)
Carnivorous Death-Parrot
Cheatyface
Johnny, Combo Player
Magic Word
Mer Man
Richard Garfield, Ph.D.
Rings a Bell
Time Out
Topsy Turvy
Wall of Fortune
Acornelia, Fashionable Filcher
Dirty Rat
Duh
Enter the Dungeon
Hoisted Hireling
Infernal Spawn of Infernal Spawn of Evil
Infernius Spawnington III, Esq.
Inhumaniac
Skull Saucer
Snickering Squirrel
Stinging Scorpion
Abstract Iguanart
Blast from the Past
Boomstacker
Common Iguana
Goblin Haberdasher
Goblin S.W.A.T. Team
Infinity Elemental
Painiac
Six-y Beast
Stet, Draconic Proofreader
Super-Duper Death Ray
Yet Another Aether Vortex
B-I-N-G-O
Mother Kangaroo
Pippa, Duchess

Sawtooth Ogre
Aboroth
Arctic Wolves
Barishi
Blossoming Wreath
Briar Shield
Choking Vines
Downdraft
Fallow Wurm
Fungus Elemental
Liege of the Hollows
Llanowar Druid
Mwonvuli Ooze
Nature's Kiss
Striped Bears
Sylvan Hierophant
Uktabi Efreet
Battle Hurda
Fledgling Griffin
Join the Ranks
Kitesail Apprentice
Marsh Threader
Perimeter Captain
Refraction Trap
Rest for the Weary
Ruin Ghost
Talus Paladin
Terra Eternal
Calcite Snapper
Goliath Sphinx
Halimar Excavator
Horizon Drake
Mysteries of the Deep
Permafrost Trap
Quest for Ula's Temple
Sejiri Merfolk
Selective Memory
Spell Contortion
Thada Adel, Acquisitor
Tideforce Elemental
Vapor Snare
Voyager Drake
Wind Zendikon
Agadeem Occultist
Bojuka Brigand
Caustic Crawler
Corrupted Zendikon
Dead Reckoning
Jagwasp Swarm
Quest for the Nihil Stone
Scrib Nibblers
Shoreline Salvager
Tomb Hex
Akoum Battlesinger
Bazaar Trader
Bull Rush
Crusher Zendikon
Grotag Thrasher
Quest for the Goblin Lord
Ricochet Trap
Roiling Terrain
Rumbling Aftershocks
Stone Idol Tra

In [100]:
lemmatized_card_texts

{"Ancestor's Chosen": '<card> <line> <sentence> first strike this creature deals combat damage before creatures without first strike </sentence> </line> <line> <sentence> when <CARD_NAME> enters the battlefield, you gain 1 life for each card in your graveyard </sentence> </line> </card>',
 'Angel of Mercy': '<card> <line> flying </line> <line> <sentence> when <CARD_NAME> enters the battlefield, you gain 3 life </sentence> </line> </card>',
 'Aven Cloudchaser': "<card> <line> <sentence> flying this creature can't be blocked except by creatures with flying or reach </sentence> </line> <line> <sentence> when <CARD_NAME> enters the battlefield, destroy target enchantment </sentence> </line> </card>",
 'Ballista Squad': '<card> <line> <precolon> <color> W,X </color> <color> T </color> </precolon> <sentence>  <CARD_NAME> deals x damage to target attacking or blocking creature </sentence> </line> </card>',
 'Bandage': '<card> <line> <sentence> prevent the next 1 damage that would be dealt to 

In [101]:
names = []
rarities = []
colors = []
card_types = []
mana_costs = []
powers = []
toughnesses = []
loyalties = []
texts = []

for name, feature_dict in cards.items():
    names.append(name)
    rarities.append(feature_dict["rarity"])
    colors.append("".join(feature_dict["colors"]))
    mana_costs.append(process_color(feature_dict["manaCost"]))
    
    card_type = feature_dict["type"]
    if "Creature" in card_type:
        card_type = "Creature"
    if "Enchantment" in card_type:
        card_type = "Enchantment"
    if "loyalty" in list(feature_dict.keys()):
        card_type = "Planeswalker"
    card_types.append(card_type)
    
    toughness = None if card_type != "Creature" else feature_dict["toughness"]
    toughnesses.append(toughness)
    power = None if card_type != "Creature" else feature_dict["power"]
    powers.append(power)
    loyalty = None if card_type != "Planeswalker" else feature_dict["loyalty"]
    loyalties.append(loyalty)
    
    texts.append(lemmatized_card_texts[name])
    
cards_df = pd.DataFrame({"name":names, "rarity":rarities, "color":colors, "type":card_types,
                         "mana_cost":mana_costs, "power":powers, "toughness":toughnesses, 
                         "loyalty":loyalties, "text":texts})

cards_df.to_csv("cards.csv", index=False)

In [102]:
cards_df['text'].values

array(['<card> <line> <sentence> first strike this creature deals combat damage before creatures without first strike </sentence> </line> <line> <sentence> when <CARD_NAME> enters the battlefield, you gain 1 life for each card in your graveyard </sentence> </line> </card>',
       '<card> <line> flying </line> <line> <sentence> when <CARD_NAME> enters the battlefield, you gain 3 life </sentence> </line> </card>',
       "<card> <line> <sentence> flying this creature can't be blocked except by creatures with flying or reach </sentence> </line> <line> <sentence> when <CARD_NAME> enters the battlefield, destroy target enchantment </sentence> </line> </card>",
       ...,
       '<card> <line> flash </line> <line> flying </line> <line> <sentence> as long as an opponent has eight or more cards in their graveyard, rogues you control get +1/+0 </sentence> </line> <line> <sentence> whenever one or more rogues you control attack, each opponent mills two cards </sentence> </line> </card>',
     

In [103]:
names = list(cards.keys())

In [77]:
colors = ["".join(cards[key]["colors"]) for key in list(cards.keys())]
colors = [color for color in colors if color != ""]
color_outputs = [[(1 if color_char in color else 0)/len(color) for color_char in ['B', 'G', 'R', 'U', 'W']] for color in colors]

In [78]:
colorful_names_and_texts = zip(names, lemmatized_card_texts)
colorful_names_and_texts = [(name, text) for name, text in colorful_names_and_texts if "".join(cards[name]["colors"]) != ""]
colorful_names, colorful_texts = [tup[0] for tup in colorful_names_and_texts], [tup[1] for tup in colorful_names_and_texts]

In [80]:
colorful_data = list(zip(colorful_names, [model[t] for t in colorful_texts], color_outputs))
colorful_test_data = random.sample(colorful_data, int(len(colorful_data)*0.3))
colorful_train_data = [d 
                       for d in colorful_data 
                       if d[0] not in set([x[0] for x in colorful_test_data])]

<ipython-input-80-d5acebe441ef>:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  colorful_data = list(zip(colorful_names, [model[t] for t in colorful_texts], color_outputs))


KeyError: "word 'Ancestor's Chosen' not in vocabulary"

In [ ]:
RUNS_THROUGH = 5

class AllColorNet(torch.nn.Module):
    
    def __init__(self, layer_sizes, rho = 0.1):
        super(AllColorNet, self).__init__()
        self.rho = rho

        layer_sizes = [[layer_sizes[i], layer_sizes[i+1]] 
                       for i in range(len(layer_sizes)-1)]
        layers = []
        for i in range(len(layer_sizes)):
            layers.append(nn.Linear(*layer_sizes[i]))
            if i != len(layer_sizes)-1:
                layers.append(nn.ReLU())
        layers.append(nn.Softmax())
    
        self.linear_layers = torch.nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.linear_layers(x)
        x = torch.transpose(x, 1, 0)
        x = torch.mean(x, dim = 1)
        return x

    def fit(self, x, y,
          loss_function = torch.nn.functional.binary_cross_entropy_with_logits,
          rho = None):
        if rho == None:
            rho = self.rho
        opt = torch.optim.Adam(self.parameters(), lr = rho)
        for t in range(RUNS_THROUGH):
            pred = self(x)
            loss = loss_function(pred, y)
            opt.zero_grad()
            loss.backward()
            opt.step()

In [ ]:
net1 = AllColorNet([75, 150, 5], rho = 0.1)
for name, x, y in tqdm(colorful_train_data):
    x, y = torch.Tensor(x), torch.Tensor(y)
    net1.fit(x, y, loss_function = torch.nn.MSELoss())
    
    """print(name)
    text = cards[name]["text"]
    print(text)
    print(x.shape)
    print(y)
    print("\n")"""

In [ ]:
for name, x, y in colorful_test_data:
    print(name)
    text = cards[name]["text"]
    x, y = torch.Tensor(x), torch.Tensor(y)
    print(text)
    y_pred = net1(x)
    print(x.shape)
    print(y)
    print(y_pred)
    print("\n")


In [ ]:
RUNS_THROUGH = 5

class SingleColorNet(torch.nn.Module):
    
    def __init__(self, layer_sizes):
        super(SingleColorNet, self).__init__()

        layer_sizes = [[layer_sizes[i], layer_sizes[i+1]] 
                       for i in range(len(layer_sizes)-1)]
        layers = []
        for i in range(len(layer_sizes)):
            layers.append(nn.Linear(*layer_sizes[i]))
            if i != len(layer_sizes)-1:
                """layers.append(nn.ReLU())"""
        layers.append(nn.Tanh())
    
        self.linear_layers = torch.nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.linear_layers(x)
        return x

    def fit(self, x, y,
          loss_function = torch.nn.functional.binary_cross_entropy_with_logits,
          rho = None):
        if rho == None:
            rho = 0.1
        opt = torch.optim.Adam(self.parameters(), lr = rho)
        for t in range(RUNS_THROUGH):
            pred = self(x)
            loss = loss_function(pred, y)
            opt.zero_grad()
            loss.backward()
            opt.step()

In [ ]:
CURRENT_COLOR = "G" #BGRUW
net2 = SingleColorNet([75, 60, 30, 1])
for name, x, y in tqdm(colorful_train_data):
    x = torch.Tensor(x)
    x = torch.transpose(x, 1, 0)
    x = torch.mean(x, dim = 1)
    old_y = y.copy()
    y = torch.tensor(1.0 
                     if CURRENT_COLOR in cards[name]["text"]
                     else -1.0)
    net2.fit(x, y, loss_function = torch.nn.MSELoss(), rho = 0.1)

In [ ]:
d = {"-1,-1":0, "-1,1":0, "1,-1":0, "1,1":0}
for name, x, y in colorful_train_data:
    print(name)
    text = cards[name]["text"]
    x = torch.Tensor(x)
    x = torch.transpose(x, 1, 0)
    x = torch.mean(x, dim = 1)
    print(y)
    y = torch.tensor(1.0 
                     if CURRENT_COLOR in cards[name]["text"]
                     else -1.0)
    y_pred = net2(x)
    d[f"{int(y.item())},{int(y_pred.item())}"]+=1
    print(x.shape)
    print(y)
    print(y_pred)
    print("\n")
d

In [ ]:
for i in range(100):
    name = str(random.choice(list(cards.keys())))
    print(name)
    print(cards[name]["text"])
    print()

In [ ]:
N = 6
keys = set()
ngram_dict = {}
for card in tqdm(lemmatized_card_texts[0:int(len(lemmatized_card_texts))]):
    if len(card) < 5:
        continue
    for i in range(len(card)-(N-1)):
        ngram = " ".join(card[i:i+N])
        if ngram not in keys:
            keys.add(ngram)
            ngram_dict[ngram] = 0
        ngram_dict[ngram]+=1

ngram_dict = [[key, ngram_dict[key]] for key in list(ngram_dict.keys())]
ngram_dict.sort(key = lambda x: x[1], reverse = True)
ngram_dict = ngram_dict[0:10]
categories, counts = [c[0] for c in ngram_dict], [c[1] for c in ngram_dict]

fig = plt.figure()
ax = fig.add_axes([0,0,2,2])
ax.set_title('N-gram Count')
ax.bar(categories, counts)
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
category_dict = {}
keys = set()
for name in list(cards.keys()):
    value = "".join(cards[name]["colors"])
    if value not in keys:
        keys.add(value)
        category_dict[value] = 0
    category_dict[value]+=1
del category_dict[""]

category_dict = {key: category_dict[key]/sum(list(category_dict.values())) for key in list(category_dict.keys()) if len(key) <= 2}
category_dict = [[key, category_dict[key]] for key in list(category_dict.keys())]
category_dict.sort(key = lambda x: x[1], reverse = True)
categories, counts = [c[0] for c in category_dict], [c[1] for c in category_dict]

fig = plt.figure()
ax = fig.add_axes([0,0,2,2])
ax.set_title('Prominence of Color')
ax.bar(categories, counts)
plt.show()

In [ ]:
category_dict = {}
keys = set()
for name in list(cards.keys()):
    value = "".join(cards[name]["rarity"])
    if value not in keys:
        keys.add(value)
        category_dict[value] = 0
    category_dict[value]+=1

category_dict = {key: category_dict[key]/sum(list(category_dict.values())) for key in list(category_dict.keys())}
category_dict = [[key, category_dict[key]] for key in list(category_dict.keys())]
category_dict.sort(key = lambda x: x[1], reverse = True)
category_dict = category_dict[0:8]
categories, counts = [c[0] for c in category_dict], [c[1] for c in category_dict]

fig = plt.figure()
ax = fig.add_axes([0,0,2,2])
ax.set_title('Prominence of Rarity')
ax.bar(categories, counts)
plt.show()

In [ ]:
category_dict = {}
keys = set()
for name in list(cards.keys()):
    value = "".join(cards[name]["type"])
    if value not in keys:
        keys.add(value)
        category_dict[value] = 0
    category_dict[value]+=1

category_dict = {key: category_dict[key]/sum(list(category_dict.values())) for key in list(category_dict.keys())}
category_dict = [[key, category_dict[key]] for key in list(category_dict.keys())]
category_dict.sort(key = lambda x: x[1], reverse = True)
category_dict = category_dict[0:8]
print(category_dict)
categories, counts = [c[0] for c in category_dict], [c[1] for c in category_dict]

fig = plt.figure()
ax = fig.add_axes([0,0,2,2])
ax.set_title('Prominence of Type')
ax.bar(categories, counts)
plt.show()

In [ ]:
for key in list(cards.keys()):
    print(key)

In [ ]:
cards["Denizen of the Deep"]["colors"]

In [ ]:
s = set()
d = {}
for key, color in [(key,str(cards[key]["rarity"])) for key in list(cards.keys())]:
    s.add(color)
    if color not in list(d.keys()):
        d[color] = 0
    d[color]+=1
print(d)

In [ ]:
for key in list(cards.keys()):
    if cards[key]["rarity"] == "rare":
        print(key)

In [ ]:
hints = set()
for key in list(cards.keys()):
    if "text" not in list(cards[key].keys()):
        continue
    text = cards[key]["text"]
    starts, ends = [], []
    for index, char in enumerate(list(text)):
        if char == "(":
            starts.append(index)
        elif char == ")":
            ends.append(index)
    for start, end in zip(starts, ends):
        hints.add(text[start+1:end])

In [ ]:
len(hints) / len(list(cards.keys()))

In [ ]:
cards["Chandra, Pyrogenius"]["manaCost"]
cards["Brown Ouphe"]

In [ ]:
print(len([x for x in model.wv.vocab]))
[x for x in model.wv.vocab]

In [ ]:
for index, (n,c,t) in enumerate(zip(names, colors, lemmatized_card_texts)):
    print(n,"\n", c, "\n", t, "\n\n")
    if index > 9:
        break

In [ ]:
[1,2,3,4,5,6]/6